## In this R notebook I use Millar et al's workflow for automatic event delineation/hysteresis index calcs using the s::can and discharge data from [Kincaid et al., 2020](https://agupubs.onlinelibrary.wiley.com/doi/10.1029/2020WR027361). 

## Here I adjust the Millar workflow to accept manual event delineations input and calculate event water yields.

- ### Data publicly available here on HydroShare: https://www.hydroshare.org/resource/85fa32a11fbb49779033934a135f54ef/

- ### This larger dataset includes the 2014-2015 discharge and nitrate data from Vaughan, M. (2017). Vermont NEWRnet stations: 2014-2015 high-frequency DOC, nitrate, and discharge data, HydroShare, http://www.hydroshare.org/resource/faac1672244c407e9c9c8644c8211fd6.

- ### Note that there is a Hungerford data gap in 2016 to adjust this code for

- ### I downloaded on 05.02.24 and put it here in this directory /home/millieginty/OneDrive/git-repos/cQ_analysis/millar2021_R_partition_hysteresis

- ### The raw data file has discharge (q m3s), NO3, and SRP with timestamp and event start/end times for each watershed. The Millar code takes just timestamp, q, and C input csvs so I separate this raw data file into just those parameters for each site over the entire time period (>400 events from 2014 to 2018, no winter events).

## I use the Kincaid 2020 events as delineated using HydRun with manual interventions.
 
 - ### Data were copied to this repo from the BREE OneDrive directory. One csv for each watershed, 2014-2018.

In [1]:
#################
# LOAD PACKAGES #
#################

library(tidyverse)
library(viridis)
library(dplyr)
library(lubridate)

###################
# SET DIRECTORIES #
###################

# Define the input and output directories

# For Kincaid data, input and output in separate directory
input_dir <- "/home/millieginty/OneDrive/git-repos/cQ_analysis/millar2021_R_separation_hysteresis/kincaid2020_hydroshare/"
output_dir <- "/home/millieginty/OneDrive/git-repos/cQ_analysis/millar2021_R_separation_hysteresis/kincaid2020_hydroshare/output/"

# functions script in main millar directory
millar_input_dir <- "/home/millieginty/OneDrive/git-repos/cQ_analysis/millar2021_R_separation_hysteresis/"

#####################
# READ IN FUNCTIONS #
#####################

# 2024-07-08 MED note: I made a new version of the Millar functions script with my modifications
source(file.path(input_dir,"cQ_functions_MED.R"))

#################
# SET SITE INFO #
#################

# Set site name
Site = "Hungerford"

# Set year if doing yearly
#Year = 2015

# Set constituent
Analyte = "NO3"

# Set catchment area
Area = 48.1 #Hungerford

#########################
# READ IN AND TIDY DATA #
#########################

# Read in raw Hydroshare data csv from Kincaid et al 2020 found at https://www.hydroshare.org/resource/85fa32a11fbb49779033934a135f54ef/
# Downloaded on 05.02.24
allInputData15Min <- read.csv(file.path(input_dir,"hydroshare_rawData.csv"))

# Rename the 'timestamp' column to 'datetime' to conform with Millar script
names(allInputData15Min)[names(allInputData15Min) == "timestamp"] <- "datetime"

# Read in the event delineation csv files. These contain start and end datetimes


# Filter the data for just the site and for the year/time range you want
# Memory issues if you try to process all the Kincaid 2014-2018 data at once
# Remove rows with missing values
Site_input <- allInputData15Min %>%
  #filter(site == Site & year(datetime) == Year) %>%
  filter(site == Site) %>%
  drop_na(q_cms, NO3_mgNL) %>%
  select(datetime, q_cms, conc = NO3_mgNL)

# Specify constituent in data set name
dataSetName = paste(Site,"_",Analyte,"_","2014-2018", sep="")

# Chose constitution for plot axes labels (NO3, TOC, or turbidity)
constit <- Analyte

Site_input$datetime <- as.POSIXct(Site_input$datetime,format("%Y-%m-%d %H:%M:%S"),tz="EST")

# Rescale the data
Site_input <- Site_input %>% 
  mutate(rescaled_conc = ((conc-min(conc))/(max(conc)-min(conc))*max(q_cms)))

── Attaching core tidyverse packages ────────────────────────────────────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.3     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ──────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Loading required package: viridisLite



### Running Millar batch event delineation with Kincaid parameters

- 2024-07-08 MED note: I went into cQ_functions_MED.R and changed the `stormEventCalcs` function so that a column called `tot_constit_mgNL` is created, a nitrate/DOC/any constituent analog of the `tot_q_m3` that is calculated in the original Millar script.